In [1]:
import pandas as pd
from cleaning import create_demand
import pyomo.environ as pyo
import pyomo

# Data import

In [2]:
nodes = pd.read_pickle("../../data/original/nodes.pkl")
channels = pd.read_pickle("../../data/original/channels.pkl")

## Modeling

Instead of minimizing the fee, now we want to maximise the probability of the payment succeeding on the first attempt.
Moreover, we assign a "penalization rate" for nodes that are only onion nodes.

In [3]:
pyomo.common.timing.report_timing()